##Introducción  
Se creará una red neuronal que clasifique imagenes del dataset MNIST. Dicho dataset es una recopilación de imágenes de números escritos a mano. Se tiene la intención de predecir el número que aparece en cada imagen.  
Una vez hecho esto, se aplicará un filtro de pixelado. Se espera que el filtro reduzca la precisión de la red neuronal simple.

##Importaciones

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
import numpy as np

## Carga del dataset MNIST

In [2]:
transform_original = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_original)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_original)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 492kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.4MB/s]


##Elaboración del modelo

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Entrenamiento del modelo

In [6]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


## Evaluación del modelo

In [8]:
def test_model(model, dataloader):
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

accuracy_original = test_model(model, testloader)
print(f" Precisión: {accuracy_original:.4f}")


 Precisión: 0.9894


##Aplicación de filtro de pixelado y nueva evaluación

In [9]:
class Pixelate(object):
    def __init__(self, factor=4):
        self.factor = factor

    def __call__(self, img):
        small = F.resize(img, [img.size(1) // self.factor, img.size(2) // self.factor], interpolation=transforms.InterpolationMode.BILINEAR)
        return F.resize(small, [img.size(1), img.size(2)], interpolation=transforms.InterpolationMode.NEAREST)

transform_pixelated = transforms.Compose([
    transforms.ToTensor(),
    Pixelate(factor=4)
])

testset_pixelated = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transform_pixelated)
testloader_pixelated = torch.utils.data.DataLoader(testset_pixelated, batch_size=64, shuffle=False)

accuracy_pixelated = test_model(model, testloader_pixelated)
print(f"Precisión con imágenes pixeladas: {accuracy_pixelated:.4f}")


Precisión con imágenes pixeladas: 0.7683
